In [1]:
%load_ext autoreload
%autoreload 2
import utils
import torch
import numpy as np
from torch.utils.data import DataLoader, random_split
import transformers
from torchinfo import summary
from ibm_dataset import IBMDebater
from models.audio_model import AudioModel
from train_audio import train_loop
transformers.logging.set_verbosity_error()

In [2]:
data_path = 'data/ibm_debater/full'

data = IBMDebater(data_path, 'train', max_audio_len=10, load_text=False)
train_len = int(len(data)*0.7)
data_train, data_val = random_split(data, [train_len, len(data) - train_len])

small_data_dim = 0.2
rnd_idx = np.random.choice(np.array([i for i in range(1, len(data))]), size=int(len(data)*small_data_dim))
small_data = torch.utils.data.Subset(data, rnd_idx)
train_len = int(len(small_data)*0.7) 
small_data_train, small_data_val = random_split(small_data, [train_len, len(small_data) - train_len])
print(len(small_data_train), len(small_data_val))

298 129


In [3]:
batch_size = 16
loader_train = DataLoader(small_data_train,
                    batch_size=batch_size,
                    shuffle=True,
                    collate_fn=utils.batch_generator_wav2vec,
                    drop_last=True)
loader_val = DataLoader(small_data_val,
                    batch_size=batch_size,
                    shuffle=False,
                    collate_fn=utils.batch_generator_wav2vec,
                    drop_last=True)

In [4]:
model = AudioModel(chunk_size=10, dim=32, device='cuda')
model.cuda()
summary(model)

Layer (type:depth-idx)                   Param #
AudioModel                               --
├─FeatureExtractor: 1-1                  --
│    └─ModuleList: 2-1                   --
│    │    └─ConvLayerBlock: 3-1          (6,144)
│    │    └─ConvLayerBlock: 3-2          (786,432)
│    │    └─ConvLayerBlock: 3-3          (786,432)
│    │    └─ConvLayerBlock: 3-4          (786,432)
│    │    └─ConvLayerBlock: 3-5          (786,432)
│    │    └─ConvLayerBlock: 3-6          524,288
│    │    └─ConvLayerBlock: 3-7          524,288
├─Linear: 1-2                            16,000
├─Linear: 1-3                            513
├─ReLU: 1-4                              --
Total params: 4,216,961
Trainable params: 1,065,089
Non-trainable params: 3,151,872

In [6]:
train_loop(model, loader_train=loader_train, loader_val=loader_val, epochs=4, device='cuda')

KeyboardInterrupt: 